# RidingMowers Ownership Prediction with SVM


## 1. Setup

In [1]:
# Common imports
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score
np.random.seed(1)

# 2. Load the data

In [2]:
# Uncomment the following snippet of code to debug problems with finding the .csv file path
# This snippet of code will exit the program and print the current working directory.
#import os
#print(os.getcwd())

In [3]:
RidingMowers = pd.read_csv('RidingMowers.csv')
RidingMowers.head(20)

,Income,Lot_Size,Ownership
0,60.0,18.4,Owner
1,85.5,16.8,Owner
2,64.8,21.6,Owner
3,61.5,20.8,Owner
4,87.0,23.6,Owner
5,110.1,19.2,Owner
6,108.0,17.6,Owner
7,82.8,22.4,Owner
8,69.0,20.0,Owner
9,93.0,20.8,Owner


In [4]:
RidingMowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     24 non-null     float64
 1   Lot_Size   24 non-null     float64
 2   Ownership  24 non-null     object 
dtypes: float64(2), object(1)
memory usage: 704.0+ bytes


In [5]:
RidingMowers.describe()

,Income,Lot_Size
count,24.000000,24.000000
mean,68.437500,18.950000
std,19.793144,2.428275
min,33.000000,14.000000
25%,52.350000,17.500000
50%,64.800000,19.000000
75%,83.100000,20.800000
max,110.100000,23.600000


In [6]:
# Check the missing values by summing the total na's for each variable
RidingMowers.isna().sum()

Income       0
Lot_Size     0
Ownership    0
dtype: int64

In [7]:
#Encoding the Target variable using label encoder
labelencoder = LabelEncoder()
RidingMowers['Ownership'] = labelencoder.fit_transform(RidingMowers['Ownership'])

In [8]:
RidingMowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Income     24 non-null     float64
 1   Lot_Size   24 non-null     float64
 2   Ownership  24 non-null     int32  
dtypes: float64(2), int32(1)
memory usage: 608.0 bytes


In [9]:
RidingMowers.head(27)

,Income,Lot_Size,Ownership
0,60.0,18.4,1
1,85.5,16.8,1
2,64.8,21.6,1
3,61.5,20.8,1
4,87.0,23.6,1
5,110.1,19.2,1
6,108.0,17.6,1
7,82.8,22.4,1
8,69.0,20.0,1
9,93.0,20.8,1


# Split data (train/test)

In [10]:

df_train, df_test = train_test_split(RidingMowers, test_size=0.3)
target = 'Ownership'
predictors = list(RidingMowers.columns)
predictors.remove(target)

In [11]:
X_train = df_train[predictors]
y_train = df_train[target]
X_test = df_test[predictors]
y_test = df_test[target]

## 3. Model the data

First, we will create a dataframe to hold all the results of our models.

In [12]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# 3.1 Fit a SVM classification model using linear kernal

In [13]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [14]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"linear svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# 3.2 Fit a SVM classification model using rbf kernal

In [15]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale',probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [16]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

# 3.3 Fit a SVM classification model using polynomial kernal

In [17]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10,probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [18]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## 4.0 Summary

Sorted by accuracy, the best models are:

In [19]:
performance.sort_values(by=['Accuracy'])

,model,Accuracy,Precision,Recall,F1
0,rbf svm,0.750,0.666667,0.666667,0.666667
0,poly svm,0.875,1.000000,0.666667,0.800000
0,linear svm,1.000,1.000000,1.000000,1.000000


# 6.0 predictions using Winning model

In [20]:
df_test['predicted'] = svm_lin_model.predict(X_test)
df_test.head(10)

,Income,Lot_Size,Ownership,predicted
13,52.8,20.8,0,0
18,59.4,16.0,0,0
3,61.5,20.8,1,1
14,64.8,17.2,0,0
20,47.4,16.4,0,0
17,49.2,17.6,0,0
10,51.0,22.0,1,1
4,87.0,23.6,1,1


In [21]:
df_test['pred_prob'] = svm_lin_model.predict_proba(X_test)[:,1]
df_test.head(10)

,Income,Lot_Size,Ownership,predicted,pred_prob
13,52.8,20.8,0,0,0.505951
18,59.4,16.0,0,0,0.383210
3,61.5,20.8,1,1,0.583284
14,64.8,17.2,0,0,0.474402
20,47.4,16.4,0,0,0.299999
17,49.2,17.6,0,0,0.356866
10,51.0,22.0,1,1,0.536072
4,87.0,23.6,1,1,0.842604


# Save the model to disk
Once you train a model, you want to reused it in other notebooks or applications. You can save the model to disk using the pickle module.

In [22]:
import pickle

# save model
pickle.dump(svm_lin_model, open('SVM_Linear_model_1.pkl', "wb"))
# If you wish to load this model later, simply use pickle.load method
#loaded_model = pickle.load(open('logistic_model_example01.pkl', "rb"))

### Analysis

Whatever measure to optimize for should be determined by the particular business issue and the costs of false positives and false negatives. For instance, precision might be the most crucial indicator to focus on if the cost of a false positive (forecasting a customer will buy a riding mower when they won't) is significant. On the other side, recall might be the most crucial statistic to optimize for if the cost of a false negative (predicting a buyer won't buy a riding mower when they actually will) is high. Let's assume that both false positives and false negatives have equal value if the cost of false positives and false negatives is essential for the RidingMowers model. Given that our data collection is balanced Given the limited amount of data in the file and the fact that more of it was used for training and less for testing, the accuracy score for the linear model is good and it is not overfit. As we compare accuracy with training data, it is smaller than test accuracy (82%), indicating that the linear model is not overfit.

I'm using the Linear SVM model as my winning model after taking this into account.